# CQL

In [ ]:
"""
CQL (new mate)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    
    _, tau_hat, _ = self.get_tau(policy_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]
    
    z1_presum_tau, z2_presum_tau = self._get_tensor_values(policy_obs, presum_tauom_actions, presum_tauom_tau_hat)

    z1_presum_tau = torch.logsumexp(z1_presum_tau, dim=1).mean()
    z2_presum_tau = torch.logsumexp(z2_presum_tau, dim=1).mean()
    cql_z1_loss = (z1_presum_tau - expert_z1_pred.mean()) * self.cql_weight
    cql_z2_loss = (z2_presum_tau - expert_z2_pred.mean()) * self.cql_weight
    
    zf1_loss += cql_z1_loss
    zf2_loss += cql_z2_loss

In [ ]:
"""
CQL (old mate)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    current_actions, current_log_pi = self._get_policy_actions(policy_obs, 
                                                               num_actions=self.num_presum_tauom,
                                                               network=self.policy)
    next_actions, next_log_pi = self._get_policy_actions(policy_next_obs, 
                                                         num_actions=self.num_presum_tauom,
                                                         network=self.policy)
    
    _, tau_hat, _ = self.get_tau(policy_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]

    z1_presum_tau, z2_presum_tau = self._get_tensor_values(policy_obs, presum_tauom_actions, presum_tauom_tau_hat)
    z1_current, z2_current = self._get_tensor_values(policy_obs, current_actions, presum_tauom_tau_hat)
    z1_next, z2_next = self._get_tensor_values(policy_obs, next_actions, presum_tauom_tau_hat)

    presum_tauom_density = np.log(0.5 ** current_actions.shape[-1])
    cat_z1 = torch.cat(
        [z1_presum_tau - alpha * presum_tauom_density, 
         z1_next - alpha * next_log_pi.detach(),
         z1_current - alpha * current_log_pi.detach()], 1
    )
    cat_z2 = torch.cat(
        [z2_presum_tau - alpha * presum_tauom_density, 
         z2_next - alpha * next_log_pi.detach(),
         z2_current - alpha * current_log_pi.detach()], 1
    )
    min_zf1_loss = torch.logsumexp(cat_z1, dim=1).mean()
    min_zf2_loss = torch.logsumexp(cat_z2, dim=1).mean()
    min_zf1_loss = (min_zf1_loss - expert_z1_pred.mean()) * self.min_z_weight
    min_zf2_loss = (min_zf2_loss - expert_z2_pred.mean()) * self.min_z_weight
    
    if self.with_lagrange:
        alpha_prime = torch.clamp(self.log_alpha_prime.exp(), min=0.0, max=1000000.0)
        min_zf1_loss = alpha_prime * (min_zf1_loss - self.target_action_gap)
        min_zf2_loss = alpha_prime * (min_zf2_loss - self.target_action_gap)

        self.alpha_prime_optimizer.zero_grad()
        alpha_prime_loss = (-min_zf1_loss - min_zf2_loss) * 0.5
        alpha_prime_loss.backward(retain_graph=True)
        self.alpha_prime_optimizer.step()

    zf1_loss += min_zf1_loss
    zf2_loss += min_zf2_loss

In [ ]:
        """
        CQL (std!)
        """
        if self.use_cql:
            # q1_pred = torch.sum(presum_tau * policy_z1_pred, dim=1, keepdims=True)
            # q2_pred = torch.sum(presum_tau * policy_z2_pred, dim=1, keepdims=True)
            # q1_std = presum_tau * (policy_z1_pred - q1_pred).pow(2)
            # q2_std = presum_tau * (policy_z2_pred - q2_pred).pow(2)
            # cql1_term = self.cql_weight * q1_std.sum(dim=1).sqrt().mean()
            # cql2_term = self.cql_weight * q2_std.sum(dim=1).sqrt().mean()

            cql1_term = 4.5 - self.cql_weight * policy_z1_pred.std()
            cql2_term = 4.5 - self.cql_weight * policy_z2_pred.std()
            
            zf1_loss += cql1_term
            zf2_loss += cql2_term

In [ ]:
"""
CQL (old mate; expert observations with policy actions)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    current_actions, current_log_pi = self._get_policy_actions(expert_obs, 
                                                               num_actions=self.num_presum_tauom,
                                                               network=self.policy)
    next_actions, next_log_pi = self._get_policy_actions(expert_next_obs, 
                                                         num_actions=self.num_presum_tauom,
                                                         network=self.policy)
    
    _, tau_hat, _ = self.get_tau(expert_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]

    z1_presum_tau, z2_presum_tau = self._get_tensor_values(expert_obs, presum_tauom_actions, presum_tauom_tau_hat)
    z1_current, z2_current = self._get_tensor_values(expert_obs, current_actions, presum_tauom_tau_hat)
    z1_next, z2_next = self._get_tensor_values(expert_obs, next_actions, presum_tauom_tau_hat)

    presum_tauom_density = np.log(0.5 ** current_actions.shape[-1])
    cat_z1 = torch.cat(
        [z1_presum_tau - alpha * presum_tauom_density, 
         z1_next - alpha * next_log_pi.detach(),
         z1_current - alpha * current_log_pi.detach()], 1
    )
    cat_z2 = torch.cat(
        [z2_presum_tau - alpha * presum_tauom_density, 
         z2_next - alpha * next_log_pi.detach(),
         z2_current - alpha * current_log_pi.detach()], 1
    )
    min_zf1_loss = torch.logsumexp(cat_z1, dim=1).mean() * self.cql_weight
    min_zf2_loss = torch.logsumexp(cat_z2, dim=1).mean() * self.cql_weight

    zf1_loss += min_zf1_loss
    zf2_loss += min_zf2_loss

# imports

In [1]:
import numpy as np
import torch
import gym
import csv
from rlkit.envs import make_env
from rlkit.envs.vecenv import SubprocVectorEnv, VectorEnv
from rlkit.torch.idsac.networks import QuantileMlp, Critic
from rlkit.torch.idsac.policies import TanhGaussianPolicy
from rlkit.data_management.torch_replay_buffer import TorchReplayBuffer
import rlkit.torch.pytorch_util as ptu
from rlkit.torch.core import eval_np

In [9]:
env = gym.make('Hopper-v2')
obs = env.reset()
obs.shape

(11,)

In [10]:
ob = [env.reset()] * 10
obs = torch.FloatTensor(np.array(ob))
obs.shape

torch.Size([10, 11])

In [11]:
r = torch.ones(5,).unsqueeze(1)
r.shape

torch.Size([5, 1])

In [12]:
is_expert = torch.cat([torch.ones_like(r, dtype=torch.bool),
                      torch.zeros_like(r, dtype=torch.bool)],
                      dim=0)
is_expert.shape

torch.Size([10, 1])

In [13]:
is_expert.squeeze(1)

tensor([ True,  True,  True,  True,  True, False, False, False, False, False])

In [15]:
x = obs[is_expert.squeeze(1), ...]
print(x.shape)
print(x)

torch.Size([5, 11])
tensor([[ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03]])


In [16]:
y = obs[is_expert.squeeze(1)]
print(y.shape)
print(y)

torch.Size([5, 11])
tensor([[ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03],
        [ 1.2522e+00,  2.8751e-03, -3.4778e-03,  4.0104e-03, -3.1302e-03,
          4.9540e-04,  3.7181e-03, -1.4657e-03,  1.2178e-03, -6.3569e-04,
         -1.7011e-03]])


In [ ]:
presum_tau = torch.rand(10000, 24) + 0.1
# print(presum_tau.max())
# print(presum_tau.min())

In [ ]:
presum_tau /= presum_tau.sum(axis=-1, keepdims=True)
# print(presum_tau.max())
# print(presum_tau.min())

In [ ]:
tau = torch.cumsum(presum_tau, axis=1)
tau_hat = torch.zeros_like(tau)
tau_hat[:, 0:1] = tau[:, 0:1] / 2.
tau_hat[:, 1:] = (tau[:, 1:] + tau[:, :-1]) / 2.

In [ ]:
presum_tau.mean()

In [ ]:
tau_hat.mean()

In [ ]:
th = []
pst = []

for i in range(100000):
    presum_tau = torch.rand(10, 24) + 0.1
    presum_tau /= presum_tau.sum(axis=-1, keepdims=True)
    tau = torch.cumsum(presum_tau, axis=1)
    tau_hat = torch.zeros_like(tau)
    tau_hat[:, 0:1] = tau[:, 0:1] / 2.
    tau_hat[:, 1:] = (tau[:, 1:] + tau[:, :-1]) / 2.

    pst.append(presum_tau.mean())
    th.append(tau_hat.mean())

In [ ]:
np.mean(pst)

In [ ]:
np.mean(th)

In [ ]:
1 / 0.0417